In [2]:
import json
import re
from pathlib import Path
import pandas as pd
import glob
import os

In [3]:
json_folder = Path('../data/json')
json_paths = glob.glob(str(json_folder / '*.json'))
json_paths

['../data/json/chat2023.json',
 '../data/json/chat2022.json',
 '../data/json/chat2024.json',
 '../data/json/chat2021.json']

In [4]:
def normalize_text(field):
    if isinstance(field, str):
        return field.strip()
    if isinstance(field, list):
        parts = []
        for el in field:
            if isinstance(el, str):
                parts.append(el.strip())
            elif isinstance(el, dict) and 'text' in el and isinstance(el['text'], str):
                parts.append(el['text'].strip())
        return ' '.join([p for p in parts if p])
    return ''

In [5]:
EMOJI_PATTERN = re.compile(
    r"[\U0001F300-\U0001F6FF\u2600-\u26FF\u2700-\u27BF]+",
    flags=re.UNICODE
)

def remove_emojis(text: str) -> str:
    return EMOJI_PATTERN.sub('', text)

In [6]:
rows = []
for p in json_paths:
    with open(p, 'r', encoding='utf-8') as f:
        data = json.load(f)
    msgs = data.get('messages', [])
    for m in msgs:
        text = normalize_text(m.get('text', ''))
        txt = remove_emojis(text).strip()
        if not text:
            continue
        if len(txt.split()) <= 4:
            continue
        rows.append({
            'source': os.path.splitext(os.path.basename(p))[0],
            'id': m.get('id'),
            'date': m.get('date'),
            'from':m.get('from'),
            # 'from_id':m.get('from_id'),
            'reply_to_message_id': m.get('reply_to_message_id'),
            'text': text,
        })


In [7]:
df = pd.DataFrame(rows)
df['date'] = pd.to_datetime(df['date'], errors='coerce')
df = df.sort_values('date').reset_index(drop=True)

In [8]:
df.tail()

,source,id,date,from,reply_to_message_id,text
59039,chat2024,70490,2024-09-16 17:42:52,Максим Приемка,NaN,Еще 5-ть мест на экскурсию для перваков 8 инст...
59040,chat2024,70505,2024-11-18 17:41:44,Максим Приемка,2.0,👀 Мы знаем как волнительна и тревожна для перв...
59041,chat2024,70507,2024-11-20 14:08:17,(Архив 2024) 8 институт МАИ - это IT 2024,NaN,👀 Мы знаем как волнительна и тревожна для перв...
59042,chat2024,70509,2024-11-22 10:01:41,(Архив 2024) 8 институт МАИ - это IT 2024,70507.0,8️⃣ Дорогие родители студентов 1 курса 8 инсти...
59043,chat2024,70511,2024-12-07 12:14:06,(Архив 2024) 8 институт МАИ - это IT 2024,2.0,"Вадим Кондаратцев, академический руководитель ..."


### Диалоги

In [9]:
df['global_id'] = df['source'] + '_' + df['id'].astype(int).astype(str)
df['global_parent_id'] = df.apply(lambda r: f"{r.source}_{int(r.reply_to_message_id)}"
                                           if pd.notna(r.reply_to_message_id) else None,
                                  axis=1)

In [10]:
all_gids = set(df['global_id'])

In [11]:
from collections import defaultdict
children = defaultdict(list)
for row in df.itertuples():
    gid  = row.global_id
    pgid = row.global_parent_id
    if pgid and pgid in all_gids:
        children[pgid].append(gid)

In [12]:
roots = [
    row.global_id for row in df.itertuples()
    if not row.global_parent_id or row.global_parent_id not in all_gids
]

In [13]:
def collect_paths(gid, path=None):
    path = (path or []) + [gid]
    if gid not in children or not children[gid]:
        yield path
    else:
        for child in children[gid]:
            yield from collect_paths(child, path)

dialog_paths = []
for root in roots:
    dialog_paths.extend(collect_paths(root))

In [14]:
info_map = df.set_index('global_id')[['from','id', 'text']].to_dict('index')
records = []
for thread_id, path in enumerate(dialog_paths, 1):
    plain, ann, parts = [], [], []
    for gid in path:
        rec  = info_map[gid]
        nick = rec['from']
        uid = rec['id']
        txt  = rec['text']
        plain.append(txt)
        ann.append(f"[{nick} ({uid})] {txt}")
        parts.append((nick, uid))
    records.append({
        'thread_id':        thread_id,
        'root_global_id':   path[0],
        'turns':            len(path),
        'dialog_plain':     "\n".join(plain),
        'dialog_annotated': "\n".join(ann),
        'participants':     parts,
        'global_ids':       path
    })

threads_df = pd.DataFrame(records)

In [15]:
threads_df

,thread_id,root_global_id,turns,dialog_plain,dialog_annotated,participants,global_ids
0,1,chat2021_6,1,Порядок распределения по кафедрам/направлениям...,[Институт №8 МАИ 2021 (6)] Порядок распределен...,"[(Институт №8 МАИ 2021, 6)]",[chat2021_6]
1,2,chat2021_7,1,Порядок получения общежития При зачислении мес...,[Институт №8 МАИ 2021 (7)] Порядок получения о...,"[(Институт №8 МАИ 2021, 7)]",[chat2021_7]
2,3,chat2021_10,1,Возможность заселения в одну комнату знакомых/...,[Институт №8 МАИ 2021 (10)] Возможность заселе...,"[(Институт №8 МАИ 2021, 10)]",[chat2021_10]
3,4,chat2021_11,1,Перевод между направлениями 8 института. Перев...,[Институт №8 МАИ 2021 (11)] Перевод между напр...,"[(Институт №8 МАИ 2021, 11)]",[chat2021_11]
4,5,chat2021_12,1,Программа обучения программированию на 1 курсе...,[Институт №8 МАИ 2021 (12)] Программа обучения...,"[(Институт №8 МАИ 2021, 12)]",[chat2021_12]
...,...,...,...,...,...,...,...
42977,42978,chat2024_70489,1,"👀 Студенты, а вы помните крышесносную приемную...","[Максим Приемка (70489)] 👀 Студенты, а вы помн...","[(Максим Приемка, 70489)]",[chat2024_70489]
42978,42979,chat2024_70490,1,Еще 5-ть мест на экскурсию для перваков 8 инст...,[Максим Приемка (70490)] Еще 5-ть мест на экск...,"[(Максим Приемка, 70490)]",[chat2024_70490]
42979,42980,chat2024_70505,1,👀 Мы знаем как волнительна и тревожна для перв...,[Максим Приемка (70505)] 👀 Мы знаем как волнит...,"[(Максим Приемка, 70505)]",[chat2024_70505]
42980,42981,chat2024_70507,2,👀 Мы знаем как волнительна и тревожна для перв...,[(Архив 2024) 8 институт МАИ - это IT 2024 (70...,"[((Архив 2024) 8 институт МАИ - это IT 2024, 7...","[chat2024_70507, chat2024_70509]"


In [16]:
threads_df.dialog_plain[10]

'Староста Задачи старосты\nПередавать информацию от дирекции и начальника курса, задавать им вопросы, интересующие студентов: староста - лицо группы; связываться с преподавателями, передавать от них информацию и задавать вопросы; отмечать посещаемость своей группы, передавать информацию о ней преподавателям и дирекции; посещать собрания старост. \nКак стать старостой:\nЕсть 2 исхода, как будет решено, кто будет старостой группы. 1й -- Выборы, 2й -- назначение дирекцией. Обычно старосте, со второго семестра доплачивают 670 рублей. В случае сильного желания, вы можете связаться со своим начальником курса и выразить его (желание). Основные задачи старосты: информирование группы, учет посещаемости.'

In [17]:
threads_df.query("turns == 2").head()

,thread_id,root_global_id,turns,dialog_plain,dialog_annotated,participants,global_ids
15,16,chat2021_27,2,Здравствуйте. Нужны ли какие-нибудь справки дл...,[Василий Захаров (27)] Здравствуйте. Нужны ли ...,"[(Василий Захаров, 27), (Институт №8 МАИ Чат, ...","[chat2021_27, chat2021_28]"
16,17,chat2021_29,2,"Ну лист диспансерный, форма 086-У\nСправка по ...","[Василий Захаров (29)] Ну лист диспансерный, ф...","[(Василий Захаров, 29), (Институт №8 МАИ Чат, ...","[chat2021_29, chat2021_30]"
19,20,chat2021_40,2,Ключевые даты приемной кампании на направления...,[Институт №8 МАИ 2021 (40)] Ключевые даты прие...,"[(Институт №8 МАИ 2021, 40), (Институт №8 МАИ ...","[chat2021_40, chat2021_42]"
24,25,chat2021_67,2,"Если уже подал документы, но не указал результ...","[Уля (67)] Если уже подал документы, но не ука...","[(Уля, 67), (Институт №8 МАИ Чат, 68)]","[chat2021_67, chat2021_68]"
25,26,chat2021_67,2,"Если уже подал документы, но не указал результ...","[Уля (67)] Если уже подал документы, но не ука...","[(Уля, 67), (Институт №8 МАИ Чат, 69)]","[chat2021_67, chat2021_69]"


In [19]:
threads_df.to_csv('threads.csv')

## Подготовка к тематическому моделированию

In [ ]:
! pip install spacy[transformers] rusenttokenize
! python -m spacy download ru_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.2/756.2 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.0/314.0 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 93.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 82.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import spacy
from rusenttokenize import ru_sent_tokenize
from string import punctuation

nlp = spacy.load("ru_core_news_sm")
stop_words = nlp.Defaults.stop_words

In [ ]:
def preprocess_text(text: str) -> str:
    doc = nlp(text)
    tokens = []
    for tok in doc:
        if tok.is_punct:
            tokens.append(tok.text)
            continue

        lemma = tok.lemma_.lower().strip()
        if (not lemma
            or lemma in stop_words
            or lemma.isdigit()
            or len(lemma) <= 2
           ):
            continue
        tokens.append(lemma)
    return " ".join(tokens)

threads_df['text_proc'] = threads_df['dialog_plain'].apply(preprocess_text)

In [ ]:
threads_df['text_proc'].head()

,text_proc
0,порядок распределение кафедрам / направление г...
1,порядок получение общежитие зачисление место р...
2,возможность заселение комната знакомых / друг ...
3,перевод направление институт . перевод направл...
4,программа обучение программирование курс обуче...


## Кластеризация

In [ ]:
!pip install sentence-transformers hdbscan umap-learn wordcloud bertopic[all]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.6/150.6 kB 6.9 MB/s eta 0:00:00


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
texts = threads_df['text_proc'].tolist()

### Эмбеддинги + HDBSCAN + PCA + WordCloud

In [ ]:
from itertools import islice

def chunked(iterable, n):
    it = iter(iterable)
    while True:
        chunk = list(islice(it, n))
        if not chunk:
            break
        yield chunk

In [ ]:
from langchain.embeddings.base import Embeddings
import time
import requests
from tenacity import Retrying, stop_after_attempt, wait_fixed, retry_if_exception_type

class YandexEmbeddings(Embeddings):
    def __init__(
        self,
        api_key: str,
        folder_id: str,
        sleep_interval: float = 1.0,
        retries: int = 3,
        **kwargs
    ):
        self.api_key = api_key
        self.folder_id = folder_id
        self.sleep_interval = sleep_interval
        self.retries = retries

        self.headers = {
            "Authorization": f"Api-Key {self.api_key}",
            "Content-Type":  "application/json"
        }

    def _get_model_uri(self, is_document: bool) -> str:
        mode = "doc" if is_document else "query"
        return f"emb://{self.folder_id}/text-search-{mode}/latest"

    def _embed_one(self, text: str, is_document: bool) -> list[float]:
        payload = {
            "modelUri": self._get_model_uri(is_document),
            "text":     text
        }
        last_exc = None

        for attempt in Retrying(
            stop=stop_after_attempt(self.retries),
            wait=wait_fixed(self.sleep_interval),
            retry=retry_if_exception_type(requests.RequestException)
        ):
            with attempt:
                resp = requests.post(
                    "https://llm.api.cloud.yandex.net/foundationModels/v1/textEmbedding",
                    json=payload,
                    headers=self.headers,
                    timeout=10
                )
                resp.raise_for_status()
                data = resp.json()
                if "embedding" in data:
                    return data["embedding"]
                raise ValueError(f"Нет ключа 'embedding' в ответе: {data}")

        raise RuntimeError(f"Не удалось получить embedding после {self.retries} попыток")

    def embed_query(self, text: str) -> list[float]:
        return self._embed_one(text, is_document=False)

    def embed_documents(self, texts: list[str], chunk_size: int = 15) -> list[list[float]]:
        embeddings = []
        for txt in texts:
            emb = self._embed_one(txt, is_document=True)
            embeddings.append(emb)
            # print(embeddings[:2])
            time.sleep(self.sleep_interval)
        return embeddings


In [ ]:
emb = YandexEmbeddings(api_key="", folder_id="")

In [ ]:
emb.embed_query("привет")

[0.0440673828125,
 0.03973388671875,
 -0.06744384765625,
 -0.0767822265625,
 0.038482666015625,
 0.02471923828125,
 -0.01372528076171875,
 -0.1898193359375,
 -0.0142059326171875,
 0.038360595703125,
 -0.0166778564453125,
 -0.10662841796875,
 0.051300048828125,
 -0.06787109375,
 0.0159149169921875,
 -0.0257720947265625,
 0.09423828125,
 -0.08990478515625,
 -0.01433563232421875,
 -0.0810546875,
 0.07763671875,
 0.03826904296875,
 -0.061981201171875,
 -0.037261962890625,
 -0.040252685546875,
 -0.04351806640625,
 0.065185546875,
 0.002593994140625,
 0.09149169921875,
 -0.06085205078125,
 -0.05584716796875,
 0.07086181640625,
 -0.09112548828125,
 0.08563232421875,
 -0.033660888671875,
 -0.0753173828125,
 0.018585205078125,
 0.0253753662109375,
 -0.03021240234375,
 0.061004638671875,
 -0.139892578125,
 -0.019195556640625,
 0.036895751953125,
 0.01910400390625,
 -0.019012451171875,
 0.0006270408630371094,
 0.0125732421875,
 0.037811279296875,
 0.03863525390625,
 0.06561279296875,
 -0.04165649

In [61]:
docs_vecs = emb.embed_documents(texts)

RetryError: RetryError[<Future at 0x78b9dee136d0 state=finished raised HTTPError>]

In [ ]:
threads_df['embed_texts'] = docs_vecs

In [ ]:
import numpy as np

X = np.array(docs_vecs)
print(X.shape)

In [ ]:
import hdbscan
from sklearn.preprocessing import normalize
from sklearn.decomposition import PCA
from wordcloud import WordCloud, STOPWORDS

In [ ]:
clusterer = hdbscan.HDBSCAN(min_cluster_size=15, metric='cosine')
labels = clusterer.fit_predict(X)

ValueError: Expected 2D array, got scalar array instead:
array=<__main__.YandexEmbeddings object at 0x78b9e47494d0>.
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [ ]:
threads_df['hdbscan'] = labels

In [ ]:
pca = PCA( n_components=2, random_state=42)
coords = pca.fit_transform(emb)
threads_df['pca_x'], threads_df['pca_y'] = coords[:,0], coords[:,1]

In [ ]:
plt.figure(figsize=(8,6))

palette = sns.color_palette('tab10', np.unique(labels).max()+2)
sns.scatterplot(x='pca_x', y='pca_y', hue='hdbscan',
                data=threads_df, palette=palette,
                legend='full', s=20)

plt.title("HDBSCAN")
plt.legend(bbox_to_anchor=(1,1)); plt.show()

In [ ]:
for cl in sorted(set(labels)):

    if cl == -1: continue  # шум
    texts_cl = threads_df.loc[threads_df['hdbscan']==cl, 'text_proc']

    wc = WordCloud(width=400, height=200, background_color='white',
                   stopwords=STOPWORDS).generate(" ".join(texts_cl))

    plt.figure(figsize=(4,2))
    plt.imshow(wc, interpolation='bilinear')
    plt.axis('off')
    plt.title(f"Cluster {cl} ({len(texts_cl)} docs)")
    plt.show()

### BERTopic + UMAP + c-TF-IDF

In [ ]:
from bertopic import BERTopic
from umap import UMAP

In [ ]:
umap_model = UMAP(n_neighbors=15, n_components=7, metric='cosine', random_state=42, min_dist=0.6)

In [ ]:
topic_model = BERTopic(embedding_model=emb,
                       umap_model=umap_model,
                       nr_topics="auto",
                       diversity=0.7,
                       top_n_words=20)

TypeError: BERTopic.__init__() got an unexpected keyword argument 'diversity'

In [ ]:
topics, probs = topic_model.fit_transform(texts)

In [ ]:
threads_df['bertopic'] = topics

In [ ]:
topic_info = topic_model.get_topic_info()
display(topic_info.head(10))

In [ ]:
for t in topic_info['Topic'].unique():
    if t == -1: continue
    fig = topic_model.visualize_barchart(topic=t, top_n_topics=1)
    fig.show()

In [ ]:
vis = topic_model.visualize_topics()
vis.show()